## 05. Results

Notebook to reproduce the figures<br>
The processed data can be downloaded from the BioImage Archive (https://doi.org/10.6019/S-BIAD2208)<br>
To download only the adata object: 'https://ftp.ebi.ac.uk/pub/databases/biostudies/S-BIAD2208/xenium/processed/crca_xenium.h5ad'<br>
To download only the sdata object: 'https://ftp.ebi.ac.uk/pub/databases/biostudies/S-BIAD2208/xenium/processed/crca_xenium.zarr.zip'<br>
Run this notebook in the 'xenium' conda env

In [ ]:
import numpy as np
import pandas as pd
import os
import sys
import subprocess
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import anndata as ad
import scanpy as sc
import squidpy as sq
import spatialdata as sd
import spatialdata_io as sdio
import spatialdata_plot
import liana as li
import zipfile
from plotnine import labs, theme, element_text

prjdir = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if prjdir not in sys.path:
    sys.path.append(prjdir)

n_jobs=8
sc.settings.n_jobs=n_jobs
sc.set_figure_params(dpi=300, frameon=True, vector_friendly=True, fontsize=10)

from matplotlib.colors import LinearSegmentedColormap
cmap = LinearSegmentedColormap.from_list('grey_to_blue', ['lightgrey', 'mediumblue'])

In [ ]:
%load_ext autoreload
%autoreload 2
import src.spatial_helpers.spatial as spp
import src.spatial_helpers.spatialplot as spl
import src.spatial_helpers.sc as scp
scp.set_all_seeds()

In [ ]:
processed_datadir = '../../data/xenium/processed'
resultsdir = '../../results/xenium'
os.makedirs(resultsdir, exist_ok=True)
sc.settings.figdir = resultsdir

In [ ]:
# download the processed data only
if not os.path.exists(processed_datadir):
    os.makedirs(processed_datadir, exist_ok=True)
    ftp_url = 'ftp://ftp.ebi.ac.uk/pub/databases/biostudies/S-BIAD/208/S-BIAD2208/Files/xenium/processed'
    command = [
        'wget',
        '-r',
        '--ftp-user=anonymous',
        '--ftp-password=anonymous',
        '--cut-dirs=9',
        '-nH',
        '-P', processed_datadir,
        ftp_url
    ]
    subprocess.run(command, capture_output=True, text=True)

    # unzip the downloaded data
    zarrzipfile = os.path.join(processed_datadir, 'crca_xenium.zarr.zip')
    with zipfile.ZipFile(zarrzipfile, 'r') as zip_ref:
        zip_ref.extractall(processed_datadir)

In [ ]:
# load sdata (for spatial plots) or just the adata object
sdata = sd.read_zarr(os.path.join(processed_datadir, 'crca_xenium.zarr')) 
adata = sc.read_h5ad(os.path.join(processed_datadir, 'crca_xenium.h5ad'))
# adata = sdata['niches']

In [ ]:
# plotting params
dpi = 300
dpi_raster = 2000
linewidth = 0.5
fontsize = 6
pagewidth = 8.3 * 0.95
pageheight = 11.7 * 0.95
sc.set_figure_params(dpi=dpi, frameon=True, vector_friendly=True, fontsize=fontsize)

## Figure 5

### A. Spatial plot of cell types

In [ ]:
# get positions for top row
ix = sdata['niches'].obs['tissue_region'] == 'core'
spatial_df = pd.DataFrame(sdata['niches'].obsm['spatial'][ix], columns=['x', 'y'])
spatial_df['group'] = sdata['niches'].obs[ix]['patient_id'].values
group_centroids = spatial_df.groupby('group')[['x', 'y']].mean().reset_index()

ax = spl.spatialplot(sdata, obs_key='celltype', shape='cell_boundaries', xlabel='µm', ylabel='µm', title=' ',
                     table_key='niches', coordinate_system='merged_µm', legendsize=2, legend_markerscale=0.3,
                     dpi=dpi_raster, fontsize=fontsize, figsize=(pagewidth, 1.4))

# axes
xticks = range(0, 95000, 10000)
ax.set_xticks(xticks)
ax.set_xticklabels([str(int(tick/1000)) for tick in reversed(xticks)]) 
ax.set_xticklabels(ax.get_xticklabels()[::-1])
yticks = range(0, 28000, 5000)
ax.set_yticks(yticks)
ax.set_yticklabels([str(int(tick/1000)) for tick in yticks]) 
ax.set_yticklabels(ax.get_yticklabels()[::-1])
plt.xlabel('mm', fontsize=5)
plt.ylabel('')
legend = ax.get_legend()
legend.set_visible(False)

# top annotation
y_top = ax.get_ylim()[1]
y_offset = 0.01 * (y_top - ax.get_ylim()[0])
for _, row in group_centroids.iterrows():
    ax.text(row['x'], y_top + y_offset, row['group'], ha='center', va='bottom', fontsize=fontsize)

# side annotation
row_labels = ['normal', 'margin', 'core']
y_positions = [4500, 12500, 20500]
x_label_pos = ax.get_xlim()[1] * 1.02
for label, y in zip(row_labels, y_positions):
    ax.text(x_label_pos, y, label, ha='left', va='center', fontsize=fontsize, rotation=90, color='black')

plt.savefig(os.path.join(resultsdir, 'Fig_5a_Spatial_celltypes.pdf'), bbox_inches='tight', pad_inches=0.1, dpi=dpi_raster)

### B. Spatial plot of selected regions from tumor cores and margins

In [ ]:
## Core
core = spp.subset_rect(sdata, x0=76300, y0=3500, xdim=500, ydim=500, table_key='niches', coordinate_system='merged_µm')
translation = Translation([-76300, -3500], axes=("x", "y"))
set_transformation(core['cell_boundaries'], transformation=translation, to_coordinate_system='merged_µm_reset')

ax = spl.spatialplot(core, obs_key='celltype', shape='cell_boundaries',  xlabel='µm', ylabel='µm', title='Tumor core', table_key='niches',
                     coordinate_system='merged_µm_reset', fontsize=6, rasterize_shapes=False, figsize=(pagewidth*0.3, pagewidth*0.25))
yticks = range(0, 600, 100)
ax.set_yticks(yticks)
ax.set_yticklabels([str(tick) for tick in yticks]) 
ax.set_yticklabels(ax.get_yticklabels()[::-1], rotation=0)
legend = ax.get_legend()
if legend:
    legend.set_visible(False)
plt.savefig(os.path.join(resultsdir, 'Fig_5c_part_1_core.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_raster)

In [ ]:
## Margin
margin = spp.subset_rect(sdata, x0=83200, y0=11300, xdim=500, ydim=500, table_key='niches', coordinate_system='merged_µm')
translation = Translation([-83200, -11300], axes=("x", "y"))
set_transformation(margin['cell_boundaries'], transformation=translation, to_coordinate_system='merged_µm_reset')

ax = spl.spatialplot(margin, obs_key='celltype', shape='cell_boundaries', xlabel='µm', ylabel='µm', title='Invasive margin', table_key='niches',
                     coordinate_system='merged_µm_reset', fontsize=6, rasterize_shapes=False, figsize=(pagewidth*0.3, pagewidth*0.25))
yticks = range(0, 600, 100)
ax.set_yticks(yticks)
ax.set_yticklabels([str(tick) for tick in yticks]) 
ax.set_yticklabels(ax.get_yticklabels()[::-1], rotation=0)
legend = ax.get_legend()
if legend:
    legend.set_visible(False) 
plt.savefig(os.path.join(resultsdir, 'Fig_5c_part_2_margin.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_raster)

### C. Cell type composition of niches

In [ ]:
df = adata.obs
dfplot = (df.groupby(['Niche', 'celltype'], observed=True).size().unstack())
dfplot = dfplot.div(dfplot.sum(axis=1), axis=0)
with mpl.rc_context({'legend.markerscale': 0.5}):
    fig, ax = plt.subplots(figsize=(pagewidth*0.5, pagewidth*0.22))
    dfplot.plot(kind="barh", stacked=True, ax=ax, width=0.7, color=spl.get_obs_colors(sdata['niches'], obs_key='celltype', colors_key='colors'))
    legend = ax.legend(bbox_to_anchor=(1, 1), labelspacing=0.15, frameon=False, loc="upper left")
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.grid(False)
    ax.set_xlim(0,1)
    ax.invert_yaxis()
    ax.tick_params(axis='both', which='major')
    plt.tight_layout()
    plt.savefig(os.path.join(resultsdir, 'Fig_5b_nichecompass_composition.pdf'), dpi=dpi_raster, bbox_inches='tight')
    plt.savefig(os.path.join(resultsdir, 'Fig_5b_nichecompass_composition.jpg'), dpi=dpi_raster, bbox_inches='tight')

### D. Markers of neutrophils in the neutrophil niche

In [ ]:
celltype = 'Neutrophil'
adata_sub = adata[(adata.obs['celltype']==celltype),:].copy()
test_genes = spp.filter_bg_genes(adata, list(adata.var.index), target_celltype='Neutrophil', expr_thres=0.4, corr_thres=0.5)
adata_sub = adata_sub[:,test_genes]

# wilcoxon
sc.tl.rank_genes_groups(adata_sub, layer='norm', groupby='Niche', key_added='Niche_rank', method='wilcoxon')

de_genes = scp.marker_dotplot(adata_sub, n_genes=5, group_by='Niche', group='Neutrophil niche', rank_key='Niche_rank',
                              swap_axes=False, font_size=6, dpi=dpi, figsize=(pagewidth*0.3, pagewidth*0.25),
                              save=os.path.join(resultsdir, 'Fig_5d_Neutrophil_niche_'+celltype+'.jpg'))
de_genes = scp.marker_dotplot(adata_sub, n_genes=5, group_by='Niche', group='Neutrophil niche', rank_key='Niche_rank',
                              swap_axes=False, font_size=6, dpi=dpi, figsize=(pagewidth*0.3, pagewidth*0.25),
                              save=os.path.join(resultsdir, 'Fig_5d_Neutrophil_niche_'+celltype+'.pdf'))

# deseq2
adata_sub.obs['Niche_redef'] = 'other'
adata_sub.obs.loc[adata_sub.obs['Niche'] == 'Neutrophil niche', 'Niche_redef'] = 'Neutrophil niche'
contrasts={'Niche': ['Niche_redef', 'Neutrophil niche', 'other']}
res, dds = scp.run_pydeseq2(adata_sub, formula='~ *CONTRAST* + tissue_region + patient_id', contrasts=contrasts, aggregate_by='name', min_cells=50, layer='counts', n_jobs=n_jobs)
res['Niche'][res['Niche']['log2FoldChange'] > 1.5]

### E. Markers of fibroblasts in the neutrophil niche

In [ ]:
celltype = 'Fibroblast'
adata_sub = adata[(adata.obs['celltype']==celltype),:].copy()
test_genes = spp.filter_bg_genes(adata, list(adata.var.index), target_celltype='Fibroblast', expr_thres=0.25, corr_thres=0.5)
adata_sub = adata_sub[:,test_genes]

# wilcoxon
sc.tl.rank_genes_groups(adata_sub, layer='norm', groupby='Niche', key_added='Niche_rank', method='wilcoxon')

de_genes = scp.marker_dotplot(adata_sub, n_genes=5, group_by='Niche', group='Neutrophil niche', rank_key='Niche_rank',
                              swap_axes=False, font_size=6, dpi=dpi, figsize=(pagewidth*0.3, pagewidth*0.25),
                              save=os.path.join(resultsdir, 'Fig_5e_Neutrophil_niche_'+celltype+'.jpg'))
de_genes = scp.marker_dotplot(adata_sub, n_genes=5, group_by='Niche', group='Neutrophil niche', rank_key='Niche_rank',
                              swap_axes=False, font_size=6, dpi=dpi, figsize=(pagewidth*0.3, pagewidth*0.25),
                              save=os.path.join(resultsdir, 'Fig_5e_Neutrophil_niche_'+celltype+'.pdf'))

# deseq2
adata_sub.obs['Niche_redef'] = 'other'
adata_sub.obs.loc[adata_sub.obs['Niche'] == 'Neutrophil niche', 'Niche_redef'] = 'Neutrophil niche'
contrasts={'Niche': ['Niche_redef', 'Neutrophil niche', 'other']}
res, dds = scp.run_pydeseq2(adata_sub, formula='~ *CONTRAST* + tissue_region + patient_id', contrasts=contrasts, aggregate_by='name', min_cells=50, layer='counts', n_jobs=n_jobs)
res['Niche'][res['Niche']['log2FoldChange'] > 1.5]

### F. Selected neutrophil/fibroblast markers in all main cell types of the neutrophil niche

In [ ]:
# genes that are upregulated in fibroblasts vs. other cell types within neutrophil niche
adata_nn = adata[(adata.obs['Niche']=='Neutrophil niche') & (adata.obs['celltype'].isin(['Neutrophil', 'Cancer cell', 'Fibroblast', 'Macrophage'])),:].copy()

# wilcoxon
sc.tl.rank_genes_groups(adata_nn, layer='norm', groupby='celltype', key_added='Niche_rank', method='wilcoxon')
sc.get.rank_genes_groups_df(adata_nn, group='Fibroblast', key='Niche_rank')

# deseq2
test_genes = spp.filter_bg_genes(adata, list(adata.var.index), target_celltype='Fibroblast', expr_thres=0.25, corr_thres=0.5)
adata_sub = adata_nn[:,test_genes].copy()
adata_sub.obs['ct_fibroblast'] = 'other'
adata_sub.obs.loc[adata_sub.obs['celltype'] == 'Fibroblast', 'ct_fibroblast'] = 'Fibroblast'
contrasts={'Fibroblast': ['ct_fibroblast', 'Fibroblast', 'other']}
res, dds = scp.run_pydeseq2(adata_sub, formula='~ *CONTRAST* + tissue_region + patient_id', contrasts=contrasts, aggregate_by='name', min_cells=50, layer='counts', n_jobs=n_jobs)
res['Fibroblast'][res['Fibroblast']['log2FoldChange'] > 1.5]

# selected genes
genes = ['CXCL1', 'CXCL3', 'CXCL5', 'IL1A', 'IL1B', 'IL1R1']

sel_genes = scp.marker_dotplot(adata_nn, genes=genes, group_by='celltype', rank_key='Niche_rank',
                               swap_axes=False, font_size=6, dpi=dpi, figsize=(pagewidth*0.3, pagewidth*0.25),
                               save=os.path.join(resultsdir, 'Fig_5F_Neutrophil_niche_selected.jpg'))
sel_genes = scp.marker_dotplot(adata_nn, genes=genes, group_by='celltype', rank_key='Niche_rank',
                               swap_axes=False, font_size=6, dpi=dpi, figsize=(pagewidth*0.3, pagewidth*0.25),
                               save=os.path.join(resultsdir, 'Fig_5F_Neutrophil_niche_selected.pdf'))

## Supplementary Figure 5

### A. Spatial plot of neutrophils

In [ ]:
sdata['niches'].obs['is_neutrophil'] = 'Other'
sdata['niches'].obs.loc[sdata['niches'].obs['celltype'] == 'Neutrophil', 'is_neutrophil'] = 'Neutrophil'
groups = ['Neutrophil']
colors_ref=["blue", "red", "yellow"]
palette = [colors_ref[i % len(colors_ref)] for i in range(len(groups))]
palette.append("#cccccc")
groups.append("other")

ax = spl.spatialplot(sdata, obs_key='is_neutrophil', shape='cell_boundaries', xlabel='µm', ylabel='µm', title=' ', 
                     groups=groups, palette=palette,
                     table_key='niches', coordinate_system='merged_µm', legendsize=2, legend_markerscale=0.3,
                     dpi=dpi_raster, fontsize=fontsize, figsize=(pagewidth, 1.4))

# axes
xticks = range(0, 95000, 10000)
ax.set_xticks(xticks)
ax.set_xticklabels([str(int(tick/1000)) for tick in reversed(xticks)]) 
ax.set_xticklabels(ax.get_xticklabels()[::-1])
yticks = range(0, 28000, 5000)
ax.set_yticks(yticks)
ax.set_yticklabels([str(int(tick/1000)) for tick in yticks]) 
ax.set_yticklabels(ax.get_yticklabels()[::-1])
plt.xlabel('mm', fontsize=5)
plt.ylabel('')
legend = ax.get_legend()
legend.set_visible(False)

# top annotation
y_top = ax.get_ylim()[1]
y_offset = 0.01 * (y_top - ax.get_ylim()[0])
for _, row in group_centroids.iterrows():
    ax.text(row['x'], y_top + y_offset, row['group'], ha='center', va='bottom', fontsize=fontsize)

# side annotation
row_labels = ['normal', 'margin', 'core']
y_positions = [4500, 12500, 20500]
x_label_pos = ax.get_xlim()[1] * 1.02
for label, y in zip(row_labels, y_positions):
    ax.text(x_label_pos, y, label, ha='left', va='center', fontsize=fontsize, rotation=90, color='black')

plt.savefig(os.path.join(resultsdir, 'Fig_S5a_Neutrophils.pdf'), bbox_inches='tight', pad_inches=0.1, dpi=dpi_raster)

### B. Nichecompass latent space UMAP

In [ ]:
with mpl.rc_context({'legend.markerscale': 0.5}):
    fig, ax = plt.subplots(figsize=(pagewidth*0.3, pagewidth*0.25))
    niche_colors = sdata['niches'].uns['colors']['Niche']
    sc.pl.embedding(adata, 'nichecompass_umap', color='Niche', palette=niche_colors, edges_width=0.02, show=False, size=0.008, ax=ax)
    ax.set_aspect('equal')
    for spine in ax.spines.values():
        spine.set_linewidth(linewidth)
    ax.set_title('')
    plt.savefig(os.path.join(resultsdir, 'Fig_5b_nichecompass_latent_umap_niches.pdf'), dpi=dpi_raster, bbox_inches='tight')
    plt.savefig(os.path.join(resultsdir, 'Fig_5b_nichecompass_latent_umap_niches.jpg'), dpi=dpi_raster, bbox_inches='tight')

### C. Neutrophil clustering

In [ ]:
df = adata.obs.copy()
df['Neutrophil'] = df['Neutrophil'].map({'in cluster': 'clustered', 'not in cluster': 'dispersed'})
clust_colors = {'clustered': '#60947d', 'dispersed': '#95b6e8'}
dfplot = (df.groupby(['Niche', 'Neutrophil'], observed=True).size().unstack())
dfplot = dfplot[['clustered', 'dispersed']]
dfplot = dfplot.div(dfplot.sum(axis=1), axis=0)
with mpl.rc_context({'legend.markerscale': 0.5}):
    fig, ax = plt.subplots(figsize=(pagewidth*0.4, pagewidth*0.27))
    dfplot.plot(kind="barh", stacked=True, ax=ax, color=clust_colors, width=0.7)
    legend = ax.legend(bbox_to_anchor=(0, 1.3), labelspacing=0.15, frameon=False, loc="upper left")
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.grid(False)
    ax.set_xlim(0,1)
    ax.invert_yaxis()
    ax.tick_params(axis='both', which='major')
    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    plt.tight_layout()
    plt.savefig(os.path.join(resultsdir, 'Fig_S5c_neutrophil_clustering.pdf'), dpi=dpi_raster, bbox_inches='tight')
    plt.savefig(os.path.join(resultsdir, 'Fig_S5c_neutrophil_clustering.jpg'), dpi=dpi_raster, bbox_inches='tight')

### D. Spatial plot of niches

In [ ]:
ax = spl.spatialplot(sdata, obs_key='Niche', shape='cell_boundaries', xlabel='µm', ylabel='µm', title=' ',
                     table_key='niches', coordinate_system='merged_µm', legendsize=2, legend_markerscale=0.3,
                     dpi=dpi_raster, fontsize=fontsize, figsize=(pagewidth, 1.4))

# axes
xticks = range(0, 95000, 10000)
ax.set_xticks(xticks)
ax.set_xticklabels([str(int(tick/1000)) for tick in reversed(xticks)]) 
ax.set_xticklabels(ax.get_xticklabels()[::-1])
yticks = range(0, 28000, 5000)
ax.set_yticks(yticks)
ax.set_yticklabels([str(int(tick/1000)) for tick in yticks]) 
ax.set_yticklabels(ax.get_yticklabels()[::-1])
plt.xlabel('mm', fontsize=5)
plt.ylabel('')
legend = ax.get_legend()
legend.set_visible(False)

# top annotation
y_top = ax.get_ylim()[1]
y_offset = 0.01 * (y_top - ax.get_ylim()[0])
for _, row in group_centroids.iterrows():
    ax.text(row['x'], y_top + y_offset, row['group'], ha='center', va='bottom', fontsize=fontsize)

# side annotation
row_labels = ['normal', 'margin', 'core']
y_positions = [4500, 12500, 20500]
x_label_pos = ax.get_xlim()[1] * 1.02
for label, y in zip(row_labels, y_positions):
    ax.text(x_label_pos, y, label, ha='left', va='center', fontsize=fontsize, rotation=90, color='black')

plt.savefig(os.path.join(resultsdir, 'Fig_S5b_Niches.pdf'), bbox_inches='tight', pad_inches=0.1, dpi=dpi_raster)

### E. Markers of cancer cells in the neutrophil niche

In [ ]:
celltype = 'Cancer cell'
adata_sub = adata[(adata.obs['celltype']==celltype),:].copy()
test_genes = spp.filter_bg_genes(adata, list(adata.var.index), target_celltype=celltype, expr_thres=0.2, corr_thres=0.6)
adata_sub = adata_sub[:,test_genes]

# wilcoxon
sc.tl.rank_genes_groups(adata_sub, layer='norm', groupby='Niche', key_added='Niche_rank', method='wilcoxon')

de_genes = scp.marker_dotplot(adata_sub, n_genes=5, group_by='Niche', group='Neutrophil niche', rank_key='Niche_rank',
                              swap_axes=False, font_size=6, dpi=dpi, figsize=(pagewidth*0.3, pagewidth*0.25),
                              save=os.path.join(resultsdir, 'Fig_S5e_Neutrophil_niche_'+celltype+'.jpg'))
de_genes = scp.marker_dotplot(adata_sub, n_genes=5, group_by='Niche', group='Neutrophil niche', rank_key='Niche_rank',
                              swap_axes=False, font_size=6, dpi=dpi, figsize=(pagewidth*0.3, pagewidth*0.25),
                              save=os.path.join(resultsdir, 'Fig_S5e_Neutrophil_niche_'+celltype+'.pdf'))

# deseq2
adata_sub.obs['Niche_redef'] = 'other'
adata_sub.obs.loc[adata_sub.obs['Niche'] == 'Neutrophil niche', 'Niche_redef'] = 'Neutrophil niche'
contrasts={'Niche': ['Niche_redef', 'Neutrophil niche', 'other']}
res, dds = scp.run_pydeseq2(adata_sub, formula='~ *CONTRAST* + tissue_region + patient_id', contrasts=contrasts, aggregate_by='name', min_cells=50, layer='counts', n_jobs=n_jobs)
res['Niche'][res['Niche']['log2FoldChange'] > 1.5]

### F. LIANA dotplot of interactions of fibroblasts (source) with other celltypes (targets) in the neutrophil niche

In [ ]:
niche = 'Neutrophil niche'
adata_nn = adata[adata.obs['Niche'] == niche].copy()
li.mt.rank_aggregate(adata_nn,
                     groupby='celltype',
                     key_added = 'liana_celltype_' + niche,
                     resource_name='consensus',
                     expr_prop=0.1,
                     min_cells = 100,
                     use_raw=False,
                     layer='norm',
                     n_perms=100000,
                     n_jobs=n_jobs,
                     verbose=True)

In [ ]:
celltype='Fibroblast'
fig = li.pl.dotplot(adata = adata_nn,
                    uns_key='liana_celltype_' + niche,
                    colour='magnitude_rank',
                    size='specificity_rank',
                    inverse_size=True,
                    inverse_colour=True,
                    source_labels=[celltype],
                    top_n=15,
                    cmap='cool',
                    orderby='magnitude_rank',
                    orderby_ascending=True,
                    figure_size=(pagewidth*0.3, pagewidth*0.25),
                    filter_fun=lambda x: x['specificity_rank'] <= 0.1,
                    return_fig=True
                   )
fig = fig + labs(x='', y='', title='') + theme(text=element_text(color='black', size=10, weight='normal'), axis_text_x=element_text(color='black', size=10, weight='normal'))
fig.save(os.path.join(resultsdir, 'Fig_S5f_Liana_' + niche + '_' + celltype + '.jpg'), width=pagewidth*0.7, height=pagewidth*0.6, dpi=dpi)
fig.save(os.path.join(resultsdir, 'Fig_S5f_Liana_' + niche + '_' + celltype + '.pdf'), width=pagewidth*0.7, height=pagewidth*0.6, dpi=dpi)